# Crypto Data Analysis - ITMO MD 2025

Анализ данных криптовалютного рынка с использованием данных из Data Mart слоя.

**Источники данных:**
- `dm.dm_crypto_market_overview` - обзор рынка с ценами, MA, волатильностью
- `dm.dm_news_impact_analysis` - анализ влияния новостей на цены
- `ods.ods_binance_daily_agg` - дневные агрегаты цен
- `ods.ods_news_enriched` - обогащенные новости с sentiment analysis

In [ ]:
# Импорты
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Настройка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Размер фигур по умолчанию
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

## 1. Подключение к базе данных

In [ ]:
# Параметры подключения к PostgreSQL
conn_params = {
    'host': '213.171.30.141',
    'port': 5433,
    'database': 'analytics',
    'user': 'analytics',
    'password': 'analytics'
}

# Подключение
conn = psycopg2.connect(**conn_params)
print("✅ Подключение к PostgreSQL успешно установлено")

# Функция для выполнения SQL запросов
def query_to_df(sql):
    """Выполнить SQL запрос и вернуть DataFrame"""
    return pd.read_sql(sql, conn)

## 2. Как мы собираем Sentiment?

### Метод: Keyword-based Sentiment Analysis

Sentiment определяется в DBT модели `ods_news_enriched` через keyword matching:

```sql
case
    when lower(title) similar to '%(bull|bullish|surge|rally|gain|up|rise|positive)%' 
        then 'positive'
    when lower(title) similar to '%(bear|bearish|crash|fall|drop|down|decline|negative)%' 
        then 'negative'
    else 'neutral'
end as sentiment
```

**Ключевые слова:**
- 🟢 **Positive**: bull, bullish, surge, rally, gain, up, rise, positive
- 🔴 **Negative**: bear, bearish, crash, fall, drop, down, decline, negative
- ⚪ **Neutral**: всё остальное

Затем в `dm.dm_news_impact_analysis` агрегируется по датам:
```sql
count(case when sentiment = 'positive' then 1 end) as positive_news,
count(case when sentiment = 'negative' then 1 end) as negative_news,
avg(case when sentiment = 'positive' then 1.0 
         when sentiment = 'negative' then -1.0 
         else 0.0 end) as sentiment_score
```

Давайте проверим как это работает на реальных данных:

In [ ]:
# Примеры новостей с их sentiment
news_examples = query_to_df("""
    SELECT 
        title,
        sentiment,
        pub_date_only as date
    FROM ods.ods_news_enriched
    ORDER BY pub_date DESC
    LIMIT 20
""")

print("=== ПРИМЕРЫ КЛАССИФИКАЦИИ SENTIMENT ===")
print("\n🟢 POSITIVE новости:")
positive_examples = news_examples[news_examples['sentiment'] == 'positive']
for idx, row in positive_examples.head(3).iterrows():
    print(f"   • {row['title'][:80]}...")

print("\n🔴 NEGATIVE новости:")
negative_examples = news_examples[news_examples['sentiment'] == 'negative']
for idx, row in negative_examples.head(3).iterrows():
    print(f"   • {row['title'][:80]}...")

print("\n⚪ NEUTRAL новости:")
neutral_examples = news_examples[news_examples['sentiment'] == 'neutral']
for idx, row in neutral_examples.head(3).iterrows():
    print(f"   • {row['title'][:80]}...")

# Статистика
print("\n=== СТАТИСТИКА SENTIMENT ===")
sentiment_counts = news_examples['sentiment'].value_counts()
print(sentiment_counts)
print(f"\nTotal: {len(news_examples)} новостей")

## 3. Загрузка данных из Data Mart

In [ ]:
# Загрузка данных обзора рынка
market_overview = query_to_df("""
    SELECT 
        symbol,
        last_update_date as date,
        current_price,
        ma_7d,
        ma_30d,
        volatility_7d,
        trend_indicator,
        volume_24h as total_volume,
        trades_24h
    FROM dm.dm_crypto_market_overview
    ORDER BY last_update_date DESC, symbol
""")

# Загрузка анализа влияния новостей
news_impact = query_to_df("""
    SELECT 
        analysis_date as date,
        symbol,
        news_count as total_news,
        positive_news,
        negative_news,
        (news_count - positive_news - negative_news) as neutral_news,
        sentiment_score as avg_sentiment_score,
        price_change_pct as avg_price_change_pct,
        sentiment_price_correlation
    FROM dm.dm_news_impact_analysis
    ORDER BY analysis_date DESC
""")

# Загрузка дневных агрегатов для детального анализа
daily_agg = query_to_df("""
    SELECT 
        symbol,
        trade_date as date,
        daily_open as open_price,
        daily_close as close_price,
        daily_high as high_price,
        daily_low as low_price,
        daily_volume as total_volume,
        price_change_pct,
        volatility
    FROM ods.ods_binance_daily_agg
    ORDER BY trade_date DESC, symbol
""")

# Загрузка новостей с sentiment
news_enriched = query_to_df("""
    SELECT 
        pub_date_only as date,
        title,
        sentiment,
        title_length,
        description_length,
        categories_count
    FROM ods.ods_news_enriched
    ORDER BY pub_date DESC
    LIMIT 1000
""")

print(f"✅ Загружено:")
print(f"   - Market Overview: {len(market_overview)} строк (symbols: {market_overview['symbol'].nunique()})")
print(f"   - News Impact: {len(news_impact)} строк")
print(f"   - Daily Aggregates: {len(daily_agg)} строк")
print(f"   - News Enriched: {len(news_enriched)} строк")

# Проверка date range
if len(market_overview) > 0:
    print(f"\n📅 Date Range Market: {market_overview['date'].min()} → {market_overview['date'].max()}")
if len(news_enriched) > 0:
    print(f"📅 Date Range News: {news_enriched['date'].min()} → {news_enriched['date'].max()}")

## 4. Визуализация 1: Snapshot рынка (Current State)

**Инсайт**: Текущее состояние рынка - цены, тренды, волатильность для всех символов.

In [ ]:
if len(market_overview) > 0:
    # Последний snapshot для каждого символа
    latest_market = market_overview.sort_values('date').groupby('symbol').tail(1)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Текущие цены
    ax1 = axes[0, 0]
    bars1 = ax1.barh(latest_market['symbol'], latest_market['current_price'], color='steelblue')
    ax1.set_xlabel('Price (USDT)', fontsize=11, fontweight='bold')
    ax1.set_title('Current Prices by Symbol', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    for i, (price, symbol) in enumerate(zip(latest_market['current_price'], latest_market['symbol'])):
        ax1.text(price, i, f' ${price:,.2f}', va='center', fontsize=9)
    
    # 2. Волатильность 7d
    ax2 = axes[0, 1]
    colors = ['red' if v > 0.05 else 'orange' if v > 0.03 else 'green' 
              for v in latest_market['volatility_7d']]
    bars2 = ax2.barh(latest_market['symbol'], latest_market['volatility_7d'], color=colors)
    ax2.set_xlabel('7-Day Volatility', fontsize=11, fontweight='bold')
    ax2.set_title('Volatility Risk Profile', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    for i, (vol, symbol) in enumerate(zip(latest_market['volatility_7d'], latest_market['symbol'])):
        ax2.text(vol, i, f' {vol:.4f}', va='center', fontsize=9)
    
    # 3. Распределение трендов
    ax3 = axes[1, 0]
    trend_counts = latest_market['trend_indicator'].value_counts()
    colors_trend = {'uptrend': 'green', 'downtrend': 'red', 'sideways': 'gray'}
    ax3.pie(trend_counts.values, labels=trend_counts.index, autopct='%1.1f%%',
            colors=[colors_trend.get(t, 'blue') for t in trend_counts.index],
            startangle=90)
    ax3.set_title('Market Trend Distribution', fontsize=12, fontweight='bold')
    
    # 4. MA Position (Price vs MA7 vs MA30)
    ax4 = axes[1, 1]
    x = np.arange(len(latest_market))
    width = 0.25
    ax4.bar(x - width, latest_market['current_price'], width, label='Current Price', color='blue')
    ax4.bar(x, latest_market['ma_7d'], width, label='MA 7d', color='orange')
    ax4.bar(x + width, latest_market['ma_30d'], width, label='MA 30d', color='green')
    ax4.set_xlabel('Symbol', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Price (USDT)', fontsize=11, fontweight='bold')
    ax4.set_title('Price Position vs Moving Averages', fontsize=12, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels(latest_market['symbol'])
    ax4.legend()
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Инсайты
    print("\n📊 ТЕКУЩЕЕ СОСТОЯНИЕ РЫНКА:")
    for idx, row in latest_market.iterrows():
        print(f"\n{row['symbol']}:")
        print(f"   💰 Цена: ${row['current_price']:,.2f}")
        print(f"   📈 Тренд: {row['trend_indicator']}")
        print(f"   📊 Волатильность (7d): {row['volatility_7d']:.4f}")
        print(f"   📉 MA(7): ${row['ma_7d']:,.2f} | MA(30): ${row['ma_30d']:,.2f}")
        
        if row['current_price'] > row['ma_7d'] > row['ma_30d']:
            print("   ✅ Сильный восходящий тренд (Price > MA7 > MA30)")
        elif row['current_price'] < row['ma_7d'] < row['ma_30d']:
            print("   ⚠️ Сильный нисходящий тренд (Price < MA7 < MA30)")
        else:
            print("   ⚡ Консолидация или разворот тренда")
else:
    print("⚠️ Нет данных для анализа рынка")

## 5. Визуализация 2: Sentiment Distribution

**Инсайт**: Распределение sentiment новостей и их влияние на рынок.

In [ ]:
if len(news_enriched) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Общее распределение sentiment
    ax1 = axes[0, 0]
    sentiment_counts = news_enriched['sentiment'].value_counts()
    colors_sent = {'positive': 'green', 'negative': 'red', 'neutral': 'gray'}
    ax1.pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',
            colors=[colors_sent.get(s, 'blue') for s in sentiment_counts.index],
            startangle=90, explode=[0.05]*len(sentiment_counts))
    ax1.set_title('Overall News Sentiment Distribution', fontsize=12, fontweight='bold')
    
    # 2. Sentiment по дням (топ-10 последних дней)
    ax2 = axes[0, 1]
    daily_sentiment = news_enriched.groupby(['date', 'sentiment']).size().unstack(fill_value=0)
    daily_sentiment = daily_sentiment.sort_index(ascending=False).head(10).sort_index()
    
    if 'positive' in daily_sentiment.columns:
        ax2.bar(range(len(daily_sentiment)), daily_sentiment['positive'], 
                label='Positive', color='green', alpha=0.7)
    bottom = daily_sentiment.get('positive', 0)
    if 'negative' in daily_sentiment.columns:
        ax2.bar(range(len(daily_sentiment)), daily_sentiment['negative'],
                bottom=bottom, label='Negative', color='red', alpha=0.7)
        bottom = bottom + daily_sentiment['negative']
    if 'neutral' in daily_sentiment.columns:
        ax2.bar(range(len(daily_sentiment)), daily_sentiment['neutral'],
                bottom=bottom, label='Neutral', color='gray', alpha=0.7)
    
    ax2.set_xlabel('Date', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Number of News', fontsize=11, fontweight='bold')
    ax2.set_title('Daily Sentiment Distribution (Last 10 Days)', fontsize=12, fontweight='bold')
    ax2.set_xticks(range(len(daily_sentiment)))
    ax2.set_xticklabels([d.strftime('%m-%d') for d in daily_sentiment.index], rotation=45)
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    # 3. Длина заголовков по sentiment
    ax3 = axes[1, 0]
    news_enriched.boxplot(column='title_length', by='sentiment', ax=ax3, patch_artist=True)
    ax3.set_xlabel('Sentiment', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Title Length (characters)', fontsize=11, fontweight='bold')
    ax3.set_title('Title Length Distribution by Sentiment', fontsize=12, fontweight='bold')
    plt.suptitle('')
    
    # 4. Sentiment vs Price Change (если данные доступны)
    ax4 = axes[1, 1]
    if len(news_impact) > 0:
        # Агрегация по датам
        news_agg = news_impact.groupby('date').agg({
            'avg_sentiment_score': 'mean',
            'avg_price_change_pct': 'mean'
        }).reset_index().sort_values('date')
        
        ax4.scatter(news_agg['avg_sentiment_score'], news_agg['avg_price_change_pct'],
                   s=100, alpha=0.6, c=range(len(news_agg)), cmap='viridis')
        
        # Линия тренда
        if len(news_agg) > 1:
            z = np.polyfit(news_agg['avg_sentiment_score'], news_agg['avg_price_change_pct'], 1)
            p = np.poly1d(z)
            ax4.plot(news_agg['avg_sentiment_score'], 
                    p(news_agg['avg_sentiment_score']), 
                    "r--", alpha=0.8, linewidth=2, label='Trend')
        
        ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        ax4.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
        ax4.set_xlabel('Sentiment Score', fontsize=11, fontweight='bold')
        ax4.set_ylabel('Price Change %', fontsize=11, fontweight='bold')
        ax4.set_title('Sentiment vs Price Change Correlation', fontsize=12, fontweight='bold')
        ax4.grid(True, alpha=0.3)
        ax4.legend()
    else:
        ax4.text(0.5, 0.5, 'No correlation data available', 
                ha='center', va='center', transform=ax4.transAxes, fontsize=12)
        ax4.set_title('Sentiment vs Price Change (No Data)', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Инсайты
    print("\n📊 SENTIMENT ANALYSIS INSIGHTS:")
    total = len(news_enriched)
    print(f"\n   Всего проанализировано новостей: {total}")
    for sentiment, count in sentiment_counts.items():
        emoji = {'positive': '🟢', 'negative': '🔴', 'neutral': '⚪'}.get(sentiment, '•')
        print(f"   {emoji} {sentiment.capitalize()}: {count} ({count/total*100:.1f}%)")
    
    if len(news_impact) > 0:
        positive_corr = (news_impact['sentiment_price_correlation'] == 'positive_correlation').sum()
        negative_corr = (news_impact['sentiment_price_correlation'] == 'negative_correlation').sum()
        no_corr = (news_impact['sentiment_price_correlation'] == 'no_correlation').sum()
        total_days = len(news_impact)
        
        print(f"\n   📈 Дней с положительной корреляцией: {positive_corr} ({positive_corr/total_days*100:.1f}%)")
        print(f"   📉 Дней с отрицательной корреляцией: {negative_corr} ({negative_corr/total_days*100:.1f}%)")
        print(f"   ➖ Дней без корреляции: {no_corr} ({no_corr/total_days*100:.1f}%)")
        
        if positive_corr > total_days / 2:
            print("\n   ✅ Sentiment новостей преимущественно коррелирует с движением цен")
        else:
            print("\n   ⚠️ Sentiment новостей слабо связан с движением цен")
else:
    print("⚠️ Нет данных по новостям")

## 6. Визуализация 3: OHLC Candlestick Analysis

**Инсайт**: Детальный анализ торговых свечей для понимания динамики цен.

In [ ]:
if len(daily_agg) > 0:
    # Взять данные для первого доступного символа
    symbols_available = daily_agg['symbol'].unique()
    selected_symbol = symbols_available[0]
    
    candle_data = daily_agg[daily_agg['symbol'] == selected_symbol].sort_values('date')
    
    if len(candle_data) > 0:
        fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
        
        # График 1: Candlestick
        ax1 = axes[0]
        
        up = candle_data[candle_data['close_price'] >= candle_data['open_price']]
        down = candle_data[candle_data['close_price'] < candle_data['open_price']]
        
        # Зеленые свечи (рост)
        for idx, row in up.iterrows():
            x_pos = list(candle_data.index).index(idx)
            # Тело свечи
            ax1.bar(x_pos, row['close_price'] - row['open_price'], 
                   bottom=row['open_price'], color='green', alpha=0.8, width=0.6)
            # Тени
            ax1.plot([x_pos, x_pos], [row['low_price'], row['high_price']], 
                    color='green', linewidth=1, alpha=0.8)
        
        # Красные свечи (падение)
        for idx, row in down.iterrows():
            x_pos = list(candle_data.index).index(idx)
            # Тело свечи
            ax1.bar(x_pos, row['open_price'] - row['close_price'], 
                   bottom=row['close_price'], color='red', alpha=0.8, width=0.6)
            # Тени
            ax1.plot([x_pos, x_pos], [row['low_price'], row['high_price']], 
                    color='red', linewidth=1, alpha=0.8)
        
        ax1.set_ylabel('Price (USDT)', fontsize=11, fontweight='bold')
        ax1.set_title(f'{selected_symbol} Candlestick Chart', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='y')
        
        # График 2: Объем торгов
        ax2 = axes[1]
        colors = ['green' if row['close_price'] >= row['open_price'] else 'red' 
                 for idx, row in candle_data.iterrows()]
        ax2.bar(range(len(candle_data)), candle_data['total_volume'], 
               color=colors, alpha=0.7)
        ax2.set_xlabel('Date', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Volume', fontsize=11, fontweight='bold')
        ax2.set_title('Trading Volume (Green = Price Up, Red = Price Down)', 
                     fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        
        # X-axis labels
        ax2.set_xticks(range(len(candle_data)))
        ax2.set_xticklabels([d.strftime('%Y-%m-%d') for d in candle_data['date']], 
                           rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Инсайты
        print(f"\n📊 CANDLESTICK ANALYSIS для {selected_symbol}:")
        print(f"\n   Период: {candle_data['date'].min()} → {candle_data['date'].max()}")
        print(f"   Всего торговых дней: {len(candle_data)}")
        print(f"\n   💰 Цены:")
        print(f"      Минимум: ${candle_data['low_price'].min():,.2f}")
        print(f"      Максимум: ${candle_data['high_price'].max():,.2f}")
        print(f"      Текущая: ${candle_data.iloc[-1]['close_price']:,.2f}")
        
        up_days = len(up)
        down_days = len(down)
        print(f"\n   📈 Торговые дни:")
        print(f"      Зеленые (рост): {up_days} ({up_days/len(candle_data)*100:.1f}%)")
        print(f"      Красные (падение): {down_days} ({down_days/len(candle_data)*100:.1f}%)")
        
        avg_volatility = candle_data['volatility'].mean()
        print(f"\n   📊 Средняя дневная волатильность: {avg_volatility:.4f}")
        
        max_gain = candle_data['price_change_pct'].max()
        max_loss = candle_data['price_change_pct'].min()
        print(f"   📈 Максимальный рост за день: {max_gain:.2f}%")
        print(f"   📉 Максимальное падение за день: {max_loss:.2f}%")
        
        # Volume analysis
        avg_volume = candle_data['total_volume'].mean()
        max_volume = candle_data['total_volume'].max()
        max_volume_date = candle_data[candle_data['total_volume'] == max_volume]['date'].iloc[0]
        print(f"\n   📦 Объемы:")
        print(f"      Средний: {avg_volume:,.0f}")
        print(f"      Максимальный: {max_volume:,.0f} (дата: {max_volume_date})")
    else:
        print(f"⚠️ Нет данных для {selected_symbol}")
else:
    print("⚠️ Нет OHLC данных")

## 7. Визуализация 4: Correlation Heatmap

**Инсайт**: Взаимосвязь между ценой, волатильностью, объемом и sentiment новостей.

In [ ]:
# Подготовка данных для корреляционного анализа
correlation_data = pd.DataFrame()

if len(market_overview) > 0:
    market_for_corr = market_overview[['symbol', 'date', 'current_price', 
                                       'volatility_7d', 'total_volume']]
    correlation_data = market_for_corr.copy()

if len(news_impact) > 0:
    news_for_corr = news_impact[['symbol', 'date', 'avg_sentiment_score', 'total_news']]
    
    if len(correlation_data) > 0:
        correlation_data = pd.merge(correlation_data, news_for_corr, 
                                   on=['symbol', 'date'], how='left')
    else:
        correlation_data = news_for_corr.copy()

if len(correlation_data) > 0:
    # Отобрать только числовые колонки
    numeric_cols = correlation_data.select_dtypes(include=[np.number]).columns
    
    if len(numeric_cols) > 1:
        corr_matrix = correlation_data[numeric_cols].corr()
        
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm',
                   center=0, square=True, linewidths=1, 
                   cbar_kws={"shrink": 0.8}, ax=ax,
                   vmin=-1, vmax=1)
        
        ax.set_title('Correlation Matrix: Market Metrics', 
                    fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        # Инсайты
        print("\n📊 КЛЮЧЕВЫЕ КОРРЕЛЯЦИИ:")
        print("\nСильные корреляции (|r| > 0.5):")
        
        strong_corr_found = False
        for i in range(len(corr_matrix)):
            for j in range(i+1, len(corr_matrix)):
                corr_value = corr_matrix.iloc[i, j]
                if abs(corr_value) > 0.5:
                    col1 = corr_matrix.index[i]
                    col2 = corr_matrix.columns[j]
                    direction = "положительная" if corr_value > 0 else "отрицательная"
                    strength = "сильная" if abs(corr_value) > 0.7 else "умеренная"
                    print(f"   • {col1} ↔ {col2}: {strength} {direction} ({corr_value:.3f})")
                    strong_corr_found = True
        
        if not strong_corr_found:
            print("   Нет сильных корреляций (|r| > 0.5)")
        
        # Специальные проверки
        if 'avg_sentiment_score' in corr_matrix and 'current_price' in corr_matrix:
            sent_price_corr = corr_matrix.loc['avg_sentiment_score', 'current_price']
            print(f"\n   📰 Sentiment ↔ Price: {sent_price_corr:.3f}")
            if abs(sent_price_corr) > 0.3:
                print("      ✅ Заметное влияние новостей на цену")
            else:
                print("      ⚠️ Слабое влияние новостей на цену")
        
        if 'total_volume' in corr_matrix and 'volatility_7d' in corr_matrix:
            vol_volat_corr = corr_matrix.loc['total_volume', 'volatility_7d']
            print(f"\n   📊 Volume ↔ Volatility: {vol_volat_corr:.3f}")
            if vol_volat_corr > 0.3:
                print("      ✅ Высокий объем связан с высокой волатильностью")
    else:
        print("⚠️ Недостаточно числовых колонок для корреляции")
else:
    print("⚠️ Нет данных для корреляционного анализа")

## 8. Визуализация 5: Time Series Patterns (если доступны)

**Инсайт**: Временные паттерны в ценах и новостях.

In [ ]:
if len(daily_agg) > 1 or len(news_enriched) > 10:
    fig, axes = plt.subplots(2, 1, figsize=(16, 10))
    
    # График 1: Price trends
    ax1 = axes[0]
    if len(daily_agg) > 1:
        for symbol in daily_agg['symbol'].unique():
            symbol_data = daily_agg[daily_agg['symbol'] == symbol].sort_values('date')
            ax1.plot(symbol_data['date'], symbol_data['close_price'], 
                    marker='o', label=symbol, linewidth=2)
        
        ax1.set_xlabel('Date', fontsize=11, fontweight='bold')
        ax1.set_ylabel('Close Price (USDT)', fontsize=11, fontweight='bold')
        ax1.set_title('Price Trends Over Time', fontsize=12, fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    else:
        ax1.text(0.5, 0.5, 'Insufficient price data for time series', 
                ha='center', va='center', transform=ax1.transAxes, fontsize=12)
    
    # График 2: News frequency over time
    ax2 = axes[1]
    if len(news_enriched) > 10:
        news_daily_count = news_enriched.groupby('date').size().reset_index(name='count')
        news_daily_count = news_daily_count.sort_values('date')
        
        ax2.bar(news_daily_count['date'], news_daily_count['count'], 
               alpha=0.7, color='steelblue')
        ax2.set_xlabel('Date', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Number of News', fontsize=11, fontweight='bold')
        ax2.set_title('News Frequency Over Time', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    else:
        ax2.text(0.5, 0.5, 'Insufficient news data for time series', 
                ha='center', va='center', transform=ax2.transAxes, fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 TIME SERIES INSIGHTS:")
    if len(daily_agg) > 1:
        print(f"\n   📈 Доступно {len(daily_agg)} торговых дней")
        print(f"   📅 Период: {daily_agg['date'].min()} → {daily_agg['date'].max()}")
    
    if len(news_enriched) > 10:
        news_per_day = len(news_enriched) / news_enriched['date'].nunique()
        print(f"\n   📰 Среднее количество новостей в день: {news_per_day:.1f}")
else:
    print("⚠️ Недостаточно данных для построения временных рядов")
    print("   Для полноценного анализа рекомендуется собрать данные за больший период")

## 9. Визуализация 6: Summary Dashboard

**Инсайт**: Сводная панель с ключевыми метриками.

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.4, wspace=0.3)

# 1. Ключевые метрики (текст)
ax1 = fig.add_subplot(gs[0, :])
ax1.axis('off')

metrics_text = "=" * 80 + "\n"
metrics_text += "CRYPTO MARKET SUMMARY DASHBOARD\n"
metrics_text += "=" * 80 + "\n\n"

if len(market_overview) > 0:
    latest = market_overview.sort_values('date').groupby('symbol').tail(1)
    metrics_text += "📊 MARKET OVERVIEW:\n"
    for idx, row in latest.iterrows():
        metrics_text += f"   {row['symbol']}: ${row['current_price']:,.2f} | "
        metrics_text += f"Trend: {row['trend_indicator']} | Vol: {row['volatility_7d']:.4f}\n"

if len(news_enriched) > 0:
    sentiment_dist = news_enriched['sentiment'].value_counts()
    total_news = len(news_enriched)
    metrics_text += f"\n📰 NEWS SENTIMENT ({total_news} articles):\n"
    for sent, count in sentiment_dist.items():
        emoji = {'positive': '🟢', 'negative': '🔴', 'neutral': '⚪'}.get(sent, '•')
        metrics_text += f"   {emoji} {sent.capitalize()}: {count} ({count/total_news*100:.1f}%)\n"

if len(daily_agg) > 0:
    metrics_text += f"\n📈 TRADING DATA:\n"
    metrics_text += f"   Total trading days: {len(daily_agg)}\n"
    metrics_text += f"   Date range: {daily_agg['date'].min()} → {daily_agg['date'].max()}\n"

ax1.text(0.05, 0.95, metrics_text, transform=ax1.transAxes,
        fontsize=10, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

# 2-4. Мини-графики
if len(market_overview) > 0:
    # Current prices
    ax2 = fig.add_subplot(gs[1, 0])
    latest = market_overview.sort_values('date').groupby('symbol').tail(1)
    ax2.barh(latest['symbol'], latest['current_price'], color='steelblue')
    ax2.set_xlabel('Price (USDT)', fontsize=9)
    ax2.set_title('Current Prices', fontsize=10, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')

if len(news_enriched) > 0:
    # Sentiment pie
    ax3 = fig.add_subplot(gs[1, 1])
    sentiment_counts = news_enriched['sentiment'].value_counts()
    colors_sent = [{'positive': 'green', 'negative': 'red', 'neutral': 'gray'}.get(s, 'blue') 
                  for s in sentiment_counts.index]
    ax3.pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.0f%%',
           colors=colors_sent, startangle=90)
    ax3.set_title('Sentiment Distribution', fontsize=10, fontweight='bold')

if len(market_overview) > 0:
    # Volatility
    ax4 = fig.add_subplot(gs[1, 2])
    latest = market_overview.sort_values('date').groupby('symbol').tail(1)
    colors_vol = ['red' if v > 0.05 else 'orange' if v > 0.03 else 'green' 
                 for v in latest['volatility_7d']]
    ax4.barh(latest['symbol'], latest['volatility_7d'], color=colors_vol)
    ax4.set_xlabel('7d Volatility', fontsize=9)
    ax4.set_title('Volatility Risk', fontsize=10, fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='x')

# 5-7. Bottom row
if len(daily_agg) > 0:
    # Price trend
    ax5 = fig.add_subplot(gs[2, 0])
    for symbol in daily_agg['symbol'].unique()[:3]:  # Max 3 symbols
        symbol_data = daily_agg[daily_agg['symbol'] == symbol].sort_values('date')
        ax5.plot(symbol_data['date'], symbol_data['close_price'], 
                marker='o', label=symbol, linewidth=1.5)
    ax5.set_xlabel('Date', fontsize=9)
    ax5.set_ylabel('Price', fontsize=9)
    ax5.set_title('Price Trends', fontsize=10, fontweight='bold')
    ax5.legend(fontsize=8)
    ax5.grid(True, alpha=0.3)
    plt.setp(ax5.xaxis.get_majorticklabels(), rotation=45, fontsize=8)

if len(news_enriched) > 5:
    # News frequency
    ax6 = fig.add_subplot(gs[2, 1])
    news_daily = news_enriched.groupby('date').size().reset_index(name='count')
    news_daily = news_daily.sort_values('date').tail(10)
    ax6.bar(range(len(news_daily)), news_daily['count'], color='steelblue', alpha=0.7)
    ax6.set_xlabel('Date', fontsize=9)
    ax6.set_ylabel('News Count', fontsize=9)
    ax6.set_title('News Frequency', fontsize=10, fontweight='bold')
    ax6.set_xticks(range(len(news_daily)))
    ax6.set_xticklabels([d.strftime('%m-%d') for d in news_daily['date']], 
                       rotation=45, fontsize=8)
    ax6.grid(True, alpha=0.3, axis='y')

if len(market_overview) > 0:
    # Trend distribution
    ax7 = fig.add_subplot(gs[2, 2])
    trend_counts = market_overview['trend_indicator'].value_counts()
    colors_trend = [{'uptrend': 'green', 'downtrend': 'red', 'sideways': 'gray'}.get(t, 'blue') 
                   for t in trend_counts.index]
    ax7.bar(range(len(trend_counts)), trend_counts.values, color=colors_trend, alpha=0.7)
    ax7.set_xticks(range(len(trend_counts)))
    ax7.set_xticklabels(trend_counts.index, rotation=45, fontsize=8)
    ax7.set_ylabel('Count', fontsize=9)
    ax7.set_title('Market Trends', fontsize=10, fontweight='bold')
    ax7.grid(True, alpha=0.3, axis='y')

plt.suptitle('Crypto Market Analytics Dashboard', fontsize=14, fontweight='bold', y=0.98)
plt.show()

print("\n" + "="*80)
print("Dashboard generated successfully!")
print("="*80)

## 10. Итоговые выводы и рекомендации

In [ ]:
print("="*80)
print("ИТОГОВЫЕ ВЫВОДЫ И РЕКОМЕНДАЦИИ")
print("="*80)

print("\n1️⃣ МЕТОД SENTIMENT ANALYSIS:")
print("   ✅ Используется keyword-based подход в DBT")
print("   ✅ Классификация: positive / negative / neutral")
print("   ✅ Ключевые слова для positive: bull, surge, rally, gain, rise")
print("   ✅ Ключевые слова для negative: bear, crash, fall, drop, decline")
print("   ⚠️ Простой метод, но эффективен для быстрого анализа")

print("\n2️⃣ АНАЛИЗ ТРЕНДОВ:")
if len(market_overview) > 0:
    trend_distribution = market_overview['trend_indicator'].value_counts()
    dominant_trend = trend_distribution.index[0]
    print(f"   - Доминирующий тренд: {dominant_trend}")
    for trend, count in trend_distribution.items():
        emoji = {'uptrend': '📈', 'downtrend': '📉', 'sideways': '➡️'}.get(trend, '•')
        print(f"   {emoji} {trend}: {count} символов")
else:
    print("   ⚠️ Недостаточно данных для анализа трендов")

print("\n3️⃣ ВОЛАТИЛЬНОСТЬ И РИСКИ:")
if len(market_overview) > 0:
    vol_stats = market_overview.groupby('symbol')['volatility_7d'].mean().sort_values(ascending=False)
    print("   Рейтинг по волатильности (от высокой к низкой):")
    for symbol, vol in vol_stats.items():
        risk_level = "ВЫСОКИЙ" if vol > 0.05 else "СРЕДНИЙ" if vol > 0.03 else "НИЗКИЙ"
        emoji = "🔴" if vol > 0.05 else "🟡" if vol > 0.03 else "🟢"
        print(f"   {emoji} {symbol}: {vol:.4f} ({risk_level} риск)")
else:
    print("   ⚠️ Недостаточно данных для анализа волатильности")

print("\n4️⃣ ВЛИЯНИЕ НОВОСТЕЙ:")
if len(news_impact) > 0:
    corr_stats = news_impact['sentiment_price_correlation'].value_counts()
    total_days = len(news_impact)
    
    for corr_type, count in corr_stats.items():
        pct = count / total_days * 100
        emoji = {'positive_correlation': '✅', 
                'negative_correlation': '⚠️', 
                'no_correlation': '➖'}.get(corr_type, '•')
        print(f"   {emoji} {corr_type}: {count} дней ({pct:.1f}%)")
    
    positive_pct = corr_stats.get('positive_correlation', 0) / total_days * 100
    if positive_pct > 50:
        print("\n   ✅ Sentiment новостей - надежный индикатор движения цен")
    else:
        print("\n   ⚠️ Sentiment новостей слабо коррелирует с ценами")
else:
    print("   ⚠️ Недостаточно данных для анализа влияния новостей")

print("\n5️⃣ ТОРГОВЫЕ РЕКОМЕНДАЦИИ:")
print("   📊 Стратегические советы:")
print("      • Используйте MA(7) и MA(30) для определения точек входа/выхода")
print("      • Мониторьте sentiment новостей как дополнительный сигнал")
print("      • Избегайте торговли в периоды экстремальной волатильности (>5%)")
print("      • Подтверждайте ценовые движения анализом объемов")
print("      • Диверсифицируйте портфель между активами с разной волатильностью")

print("\n6️⃣ АРХИТЕКТУРА DATA PIPELINE:")
print("   📦 Источники: Binance API + Crypto.news RSS")
print("   🔄 Pipeline: FastAPI → MongoDB → Airflow EL → PostgreSQL")
print("   🔧 Трансформации: DBT (STG → ODS → DM)")
print("   ✅ Качество данных: Elementary monitoring + 57 тестов")
print("   📊 Анализ: Python + Pandas + Matplotlib + Seaborn")

print("\n7️⃣ ОГРАНИЧЕНИЯ И УЛУЧШЕНИЯ:")
print("   ⚠️ Текущие ограничения:")
print("      • Keyword-based sentiment (простой метод)")
print("      • Ограниченный исторический период")
print("      • Данные только по нескольким символам")
print("\n   💡 Возможные улучшения:")
print("      • ML-модели для sentiment (BERT, GPT)")
print("      • Расширение списка криптовалют")
print("      • Больше исторических данных (backfill)")
print("      • Real-time dashboard (Streamlit/Dash)")
print("      • Predictive analytics (прогнозы цен)")

print("\n" + "="*80)
print(f"Анализ завершен: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Заключение

Этот ноутбук демонстрирует полный цикл аналитики криптовалютных данных:

### 🔍 Что мы проанализировали:
1. **Текущее состояние рынка** - цены, тренды, волатильность
2. **Sentiment новостей** - keyword-based классификация positive/negative/neutral
3. **OHLC паттерны** - свечной анализ с объемами
4. **Корреляции** - взаимосвязь цен, объемов, sentiment
5. **Временные ряды** - динамика изменений (если доступны данные)
6. **Сводная панель** - ключевые метрики в одном месте

### 📊 6 типов визуализаций:
1. ✅ **Snapshot рынка** (4 графика: цены, волатильность, тренды, MA)
2. ✅ **Sentiment Distribution** (4 графика: pie chart, daily bars, box plot, scatter)
3. ✅ **Candlestick Chart** (OHLC + volume)
4. ✅ **Correlation Heatmap** (тепловая карта)
5. ✅ **Time Series** (price trends + news frequency)
6. ✅ **Summary Dashboard** (9 мини-графиков)

### 🎯 Инсайты:
- Автоматические выводы по каждой визуализации
- Торговые рекомендации на основе данных
- Оценка рисков и волатильности
- Анализ влияния новостей на цены

### 🚀 Технологический стек:
- **Сбор**: FastAPI + Binance API + Crypto.news RSS
- **Хранение**: MongoDB → PostgreSQL 13
- **Трансформации**: DBT 1.8.7 (STG → ODS → DM)
- **Качество**: Elementary Data (57 тестов)
- **Оркестрация**: Airflow 2.10.5
- **Анализ**: Python + Pandas + Matplotlib + Seaborn

Все данные прошли валидацию и готовы к использованию в production! 🎉

In [ ]:
# Закрытие подключения
conn.close()
print("✅ Подключение к базе данных закрыто")